# Instructions for Tag-Along Project

Stability of the Grid System

Electrical grids require a balance between electricity supply and demand in order to be stable. Conventional systems achieve this balance through demand-driven electricity production. For future grids with a high share of inflexible (i.e., renewable) energy sources, the concept of demand response is a promising solution. This implies changes in electricity consumption in relation to electricity price changes. In this work, we’ll build a binary classification model to predict if a grid is stable or unstable using the UCI Electrical Grid Stability Simulated dataset.

Dataset: EGSS Data

It has 12 primary predictive features and two dependent variables.

Predictive features:

    'tau1' to 'tau4': the reaction time of each network participant, a real value within the range 0.5 to 10 ('tau1' corresponds to the supplier node, 'tau2' to 'tau4' to the consumer nodes);
    'p1' to 'p4': nominal power produced (positive) or consumed (negative) by each network participant, a real value within the range -2.0 to -0.5 for consumers ('p2' to 'p4'). As the total power consumed equals the total power generated, p1 (supplier node) = - (p2 + p3 + p4);
    'g1' to 'g4': price elasticity coefficient for each network participant, a real value within the range 0.05 to 1.00 ('g1' corresponds to the supplier node, 'g2' to 'g4' to the consumer nodes; 'g' stands for 'gamma');

Dependent variables:

    'stab': the maximum real part of the characteristic differential equation root (if positive, the system is linearly unstable; if negative, linearly stable);
    'stabf': a categorical (binary) label ('stable' or 'unstable').

Because of the direct relationship between 'stab' and 'stabf' ('stabf' = 'stable' if 'stab' <= 0, 'unstable' otherwise), 'stab' should be dropped and 'stabf' will remain as the sole dependent variable (binary classification).

Split the data into an 80-20 train-test split with a random state of “1”. Use the standard scaler to transform the train set (x_train, y_train) and the test set (x_test). Use scikit learn to train a random forest and extra trees classifier. And use xgboost and lightgbm to train an extreme boosting model and a light gradient boosting model. Use random_state = 1 for training all models and evaluate on the test set.

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("Data_for_UCI_named.csv")
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [3]:
#quick description of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


In [4]:
# dropping `stab` column 
df = df.drop('stab', axis =1)
assert 'stab' not in df.columns

In [5]:
#check distribution of target variable
df['stabf'].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [6]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [7]:
#get features and labels
features= df.drop(['stabf'], axis=1)

labels = df['stabf']

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=1)

print('X_train shape: {}'.format(x_train.shape))
print('y_train shape: {}'.format(y_train.shape))
print('X_test shape: {}'.format(x_test.shape))
print('y_test shape: {}'.format(y_test.shape))

X_train shape: (8000, 12)
y_train shape: (8000,)
X_test shape: (2000, 12)
y_test shape: (2000,)


In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [10]:
x_train_scaled = pd.DataFrame(x_train_scaled, columns = x_train.columns)
x_test_scaled = pd.DataFrame(x_test_scaled, columns = x_test.columns)

### Building Models

#### RandomForestClassifier

In [11]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state = 1)

RFC.fit(x_train_scaled, y_train)

RandomForestClassifier(random_state=1)

In [12]:
rfc_predicted = RFC.predict(x_test_scaled)

##### Measuring Model Performance for RandomForestClassifier

In [14]:
# determining the model performance via accuracy, recall, precision and f1_score

from sklearn.metrics import accuracy_score

# accuracy
accuracy = accuracy_score(y_test, rfc_predicted)
print('Accuracy: {:.4}'.format(accuracy))

from sklearn.metrics import recall_score, precision_score, f1_score, confusion_matrix, classification_report

# precision
precision = precision_score(y_test, rfc_predicted, pos_label='stable')
print('Precision: {:.4}'.format(precision))  

# recall
recall = recall_score(y_test, rfc_predicted, pos_label='stable')
print('Recall: {:.4}'.format(recall))

# F1 score
f1 = f1_score(y_test, rfc_predicted, pos_label='stable')
print('F1: {:.4}'.format(f1))

# classification report
print('\nClassification Report:\n', classification_report(y_test,rfc_predicted, digits =4))

# confusion matrix
conf_mat = confusion_matrix(y_test, rfc_predicted, labels=['unstable', 'stable'])
print('Confusion Matrix:\n', conf_mat)

Accuracy: 0.929
Precision: 0.9191
Recall: 0.8778
F1: 0.898

Classification Report:
               precision    recall  f1-score   support

      stable     0.9191    0.8778    0.8980       712
    unstable     0.9341    0.9573    0.9456      1288

    accuracy                         0.9290      2000
   macro avg     0.9266    0.9176    0.9218      2000
weighted avg     0.9288    0.9290    0.9286      2000

Confusion Matrix:
 [[1233   55]
 [  87  625]]


In [15]:
print("Training set score: {:.3f}".format(RFC.score(x_train_scaled, y_train)))
print("Test set score: {:.3f}".format(RFC.score(x_test_scaled, y_test)))


Training set score: 1.000
Test set score: 0.929


#### ExtraTreeClassifier

In [16]:
from sklearn.ensemble import ExtraTreesClassifier

ETC = ExtraTreesClassifier(random_state = 1)

#fit on the train set
ETC.fit(x_train_scaled, y_train)

ExtraTreesClassifier(random_state=1)

In [17]:
#predict on test set
etc_prediction = ETC.predict(x_test_scaled)

##### Measuring Model Performance for ExtraTreeClassifier

In [18]:
# determining the model performance via accuracy, recall, precision and f1_score
etc_accuracy = accuracy_score(y_test, etc_prediction)
print('Accuracy: {}'.format(round(etc_accuracy, 4)))

#precision
etc_precision = precision_score(y_test, etc_prediction, pos_label='stable')
print('Precision: {}'.format(round(etc_precision, 4)))  

#recall
etc_recall = recall_score(y_test, etc_prediction, pos_label='stable')
print('Recall: {}'.format(round(etc_recall, 4)))

#F1 score
etc_f1 = f1_score(y_test, etc_prediction, pos_label='stable')
print('F1: {}'.format(round(etc_f1, 4)))

#classification report
print('\nClassification Report:\n', classification_report(y_test, etc_prediction, digits =4))

#confusion matrix
etc_conf_mat = confusion_matrix(y_test, etc_prediction, labels=['unstable', 'stable'])
print('Confusion Matrix:\n', etc_conf_mat)


Accuracy: 0.928
Precision: 0.941
Recall: 0.8511
F1: 0.8938

Classification Report:
               precision    recall  f1-score   support

      stable     0.9410    0.8511    0.8938       712
    unstable     0.9218    0.9705    0.9455      1288

    accuracy                         0.9280      2000
   macro avg     0.9314    0.9108    0.9197      2000
weighted avg     0.9287    0.9280    0.9271      2000

Confusion Matrix:
 [[1250   38]
 [ 106  606]]


In [20]:
print("Training set score: {:.3f}".format(ETC.score(x_train_scaled, y_train)))
print("Test set score: {:.3f}".format(ETC.score(x_test_scaled, y_test)))


Training set score: 1.000
Test set score: 0.928


#### XGBoost

In [ ]:
from xgboost import XGBClassifier

xbc= XGBClassifier(random_state = 1)

xbc.fit(x_train_scaled, y_train)

In [ ]:
# prediction on test set
xbc_pred = xbc.predict(x_test_scaled)

##### Measuring Model Performace for XGB Classifier

In [ ]:
# determining the model performance via accuracy, recall, precision and f1_score
xbc_accuracy = accuracy_score(y_test, xbc_pred)
print('Accuracy: {}'.format(round(xbc_accuracy, 4)))
print(xbc_accuracy)
#precision
xbc_precision = precision_score(y_test, xbc_pred, pos_label='stable')
print('Precision: {}'.format(round(xbc_precision, 4)))  

#recall
xbc_recall = recall_score(y_test, xbc_pred, pos_label='stable')
print('Recall: {}'.format(round(xbc_recall, 4)))

#F1 score
xbc_f1 = f1_score(y_test, xbc_pred, pos_label='stable')
print('F1: {}'.format(round(xbc_f1, 4)))

#classification report
print('\nClassification Report:\n', classification_report(y_test, xbc_pred, digits =4))

#confusion matrix
xbc_conf_mat = confusion_matrix(y_test, xbc_pred, labels=['unstable', 'stable'])
print('Confusion Matrix:\n', xbc_conf_mat)


#### LightGBM Classifier

In [21]:
from lightgbm import LGBMClassifier

lgbm= LGBMClassifier(random_state = 1)

lgbm.fit(x_train_scaled, y_train)

LGBMClassifier(random_state=1)

In [22]:
lgbm_pred = lgbm.predict(x_test_scaled)

##### Model Performance for LGBM Classifier

In [23]:
# determining the model performance via accuracy, recall, precision and f1_score

#model accuracy
lgbm_accuracy = accuracy_score(y_test, lgbm_pred)
print('Accuracy: {}'.format(round(lgbm_accuracy, 4)))

#precision
lgbm_precision = precision_score(y_test, lgbm_pred, pos_label='stable')
print('Precision: {}'.format(round(lgbm_precision, 4)))  

#recall
lgbm_recall = recall_score(y_test, lgbm_pred, pos_label='stable')
print('Recall: {}'.format(round(lgbm_recall, 4)))

#F1 score
lgbm_f1 = f1_score(y_test, lgbm_pred, pos_label='stable')
print('F1: {}'.format(round(lgbm_f1, 4)))

#classification report
print('\nClassification Report:\n', classification_report(y_test,lgbm_pred, digits =4))

#confusion matrix
lgbm_conf_mat = confusion_matrix(y_test, lgbm_pred, labels=['unstable', 'stable'])
print('Confusion Matrix:\n', lgbm_conf_mat)


Accuracy: 0.9395
Precision: 0.9276
Recall: 0.9003
F1: 0.9138

Classification Report:
               precision    recall  f1-score   support

      stable     0.9276    0.9003    0.9138       712
    unstable     0.9458    0.9612    0.9534      1288

    accuracy                         0.9395      2000
   macro avg     0.9367    0.9307    0.9336      2000
weighted avg     0.9393    0.9395    0.9393      2000

Confusion Matrix:
 [[1238   50]
 [  71  641]]


#### Improving ExtraTreesClassifier

In [24]:
#combination of hyperparameters
n_estimators = [50, 100, 300, 500, 1000]

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None] 

hyperparameter_grid = {'n_estimators': n_estimators,

                       'min_samples_leaf': min_samples_leaf,

                       'min_samples_split': min_samples_split,

                       'max_features': max_features}




In [27]:
from sklearn.model_selection import RandomizedSearchCV


randomscv = RandomizedSearchCV(estimator = ETC, 
                              param_distributions = hyperparameter_grid, cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1, verbose = 1,
                              random_state = 1)


In [28]:
scv_fitting = randomscv.fit(x_train_scaled, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [29]:
scv_fitting.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 8,
 'max_features': None}

In [31]:
#check feature importances
scv_importance = ETC.feature_importances_

In [32]:
#print feature importances
for i,v in enumerate(scv_importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
print(scv_fitting.best_score_)

Feature: 0, Score: 0.11740
Feature: 1, Score: 0.11844
Feature: 2, Score: 0.11317
Feature: 3, Score: 0.11547
Feature: 4, Score: 0.03951
Feature: 5, Score: 0.04037
Feature: 6, Score: 0.04071
Feature: 7, Score: 0.04058
Feature: 8, Score: 0.08978
Feature: 9, Score: 0.09368
Feature: 10, Score: 0.09688
Feature: 11, Score: 0.09402
0.9241249999999999


In [33]:
x_train_scaled.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,0.367327,-0.986042,0.650447,1.547527,-0.291490,0.061535,1.293862,-0.845074,0.160918,0.339859,0.585568,0.492239
1,-0.064659,0.089437,1.035079,-1.641494,0.619865,-0.067235,-1.502925,0.486613,-0.293143,-1.558488,1.429649,-1.443521
2,-1.467850,1.298418,-0.502536,1.166046,-0.180521,0.490603,0.682560,-0.855302,1.399350,1.451534,-1.045743,0.492489
3,0.820081,0.529920,1.299657,-1.141975,-0.812854,-0.763632,1.521579,0.658780,-0.958319,1.361958,1.604140,0.275303
4,0.665424,-1.425627,0.312300,0.919137,-1.614296,0.760315,1.422019,0.639243,1.676895,0.695660,1.137504,-1.312575


In [34]:
ETC2 = ExtraTreesClassifier(max_features = None, 
                            min_samples_leaf= 8,
                            min_samples_split= 2,
                            n_estimators= 1000, 
                            random_state = 1)

#fit on train set
ETC2.fit(x_train_scaled, y_train)

ExtraTreesClassifier(max_features=None, min_samples_leaf=8, n_estimators=1000,
                     random_state=1)

In [35]:
ETC2_pred = ETC2.predict(x_test_scaled)

In [36]:
print('Classification Report:\n', classification_report(y_test,ETC2_pred, digits =4))


Classification Report:
               precision    recall  f1-score   support

      stable     0.9211    0.8694    0.8945       712
    unstable     0.9300    0.9589    0.9442      1288

    accuracy                         0.9270      2000
   macro avg     0.9256    0.9141    0.9193      2000
weighted avg     0.9268    0.9270    0.9265      2000

